In [1]:
import os
import re
import cv2
import numpy as np
from collections import defaultdict
from tqdm import tqdm

In [2]:
def parse_filename(filename):
    match = re.match(r"(\d+)_([0-9]+)\.png", filename)
    if match:
        uid = match.group(1)
        slice_num = int(match.group(2))
        return uid, slice_num
    return None, None

def process_directory(input_dir):
    files = [f for f in os.listdir(input_dir) if f.endswith('.png')]
    files_by_uid = defaultdict(list)

    # Group files by UID
    for f in files:
        uid, slice_num = parse_filename(f)
        if uid is not None:
            files_by_uid[uid].append((slice_num, f))

    for uid, slice_list in tqdm(files_by_uid.items(), desc=f"Processing {input_dir}"):
        slice_list.sort()  # Sort by slice number
        
        # Load all original grayscale images first
        original_images = {}
        for slice_num, fname in slice_list:
            path = os.path.join(input_dir, fname)
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            original_images[slice_num] = img

        # Process interior slices only
        for i in range(1, len(slice_list) - 1):
            prev_num, prev_fname = slice_list[i - 1]
            curr_num, curr_fname = slice_list[i]
            next_num, next_fname = slice_list[i + 1]

            img_prev = original_images[prev_num]
            img_curr = original_images[curr_num]
            img_next = original_images[next_num]

            stacked = np.stack([img_prev, img_curr, img_next], axis=-1)

            out_path = os.path.join(input_dir, curr_fname)
            cv2.imwrite(out_path, stacked)

        # Remove edge slices
        os.remove(os.path.join(input_dir, slice_list[0][1]))
        os.remove(os.path.join(input_dir, slice_list[-1][1]))


In [3]:
# Change this to match your directory
base_path = "/media/Datacenter_storage/Ji/brain_mri_valdo_mayo/mayo_yolo_t2s_only_rotated_GAN_3slices/images/test"
process_directory(base_path)

Processing /media/Datacenter_storage/Ji/brain_mri_valdo_mayo/mayo_yolo_t2s_only_rotated_GAN_3slices/images/test: 100%|██████████| 125/125 [02:43<00:00,  1.31s/it]
